In [1]:
!pip install openai langchain pinecone-client==2.2.4 pypdf tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.3/222.3 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.9/283.9 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.7/228.7 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [2]:
import openai
import pinecone
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
# function to read documents (gk book)
def read_doc(directory):
  loader = PyPDFDirectoryLoader(directory)
  docs = loader.load()
  return docs

In [4]:
docs = read_doc("docs/")
len(docs)

88

In [5]:
# function to split text into chunks
def chunk_data(docs, chunk_size=1000, chunk_overlap=100):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  texts = text_splitter.split_documents(docs)
  return texts

In [6]:
chunks = chunk_data(docs)
len(chunks)

156

In [7]:
# OpenAI embedding technique
embeddings = OpenAIEmbeddings(openai_api_key="")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
text = "This is a test document."
query_result = embeddings.embed_query(text)
query_result[:5]

[-0.003105442842703499,
 0.011136301333111021,
 -0.004029582553785511,
 -0.01174906406444312,
 -0.0010406979861090607]

In [24]:
# OpenAI embedding model uses 1536 dimensions
len(query_result)

1536

In [10]:
# initialize pinecone
pinecone.init(api_key="",
              environment="gcp-starter")
index_name = "langchain-vector"

In [11]:
index = Pinecone.from_documents(chunks, embeddings, index_name=index_name)

In [18]:
# retrieve cosine similarity results from vector db
def retrieve_query(query, k=2):
  matching_results = index.similarity_search(query, k=k)
  return matching_results

In [13]:
from langchain.chains.question_answering import load_qa_chain

llm = OpenAI(openai_api_key="", temperature=0.5)
chain = load_qa_chain(llm, chain_type="stuff")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [14]:
# search answers from vector db
def retrieve_answers(query):
  docsearch = retrieve_query(query)
  response = chain.run(input_documents=docsearch, question=query)
  return response

In [23]:
query = "Apa misi dari calon presiden dan wakil presiden?"
result = retrieve_answers(query)
print(result)

 Misi dari calon presiden dan wakil presiden adalah untuk melanjutkan pembangunan ekonomi, politik, kesejahteraan rakyat, dan pertahanan yang berkesinambungan, serta mencapai cita-cita abadi bangsa Indonesia yang telah dituangkan oleh para pemimpin negara sebelumnya.
